In [1]:
import numpy as np
import plotly.express as px
from numba import njit, prange, cuda
import math
import cupy as cp
import matplotlib.pyplot as plt
from functools import partial
import cv2

# Turn interactive plotting off
plt.ioff()

In [8]:
HEIGHT = 1440*1
WIDTH = 2560*1
MAX_X = .65
MIN_X = -2.35
MAX_Y = 1.25
MIN_Y = -1.25
DPI = 500

In [9]:
@njit
def test_point_trap(a, b, max_iter, dist):
    x = y = 0
    d = 1e9
    for _ in range(max_iter):
        t_x = x*x - y*y + a
        t_y = 2*x*y + b
        x = t_x
        y = t_y
        d = min(d, dist(x, y))
    return d

In [10]:
@njit
def test_point(a, b, max_iter, escaped):
    x = y = 0
    for i in range(max_iter):
        t_x = x*x - y*y + a
        t_y = 2*x*y + b
        x = t_x
        y = t_y
        if escaped(x, y):
            return i+1
    return 0

In [11]:
@njit
def escaped(x, y):
    return x*x + y*y > 4

@njit
def scale(x, min, max):
    return (max-min)*x+min

@njit(parallel=True)
def generate_fractal(arr, escaped=None, max_iter=100, dist=None):
    func = partial(test_point, escaped=escaped) if dist is None else partial(test_point_trap, dist=dist)
    for i in prange(WIDTH):
        for j in prange(HEIGHT):
            u = scale(i/(WIDTH-1), MIN_X, MAX_X)
            v = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
            arr[j, i] = func(u, v, max_iter=max_iter)

@cuda.jit
def cuda_generate_fractal(arr, max_iter):
    j, i = cuda.grid(2)
    a = scale(i/(WIDTH-1), MIN_X, MAX_X)
    b = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
    x = y = 0
    for iter in range(max_iter):
        xx = x*x
        yy = y*y
        t_x = xx - yy + a
        t_y = (x+x)*y + b
        x = t_x
        y = t_y
        if xx+yy>4:
            arr[j, i] = iter+1 -math.log(math.log(xx+yy)/math.log(2))
            return
    arr[j, i] = 0

@cuda.jit
def cuda_generate_orbit_trap_fractal(arr, max_iter, t):
    j, i = cuda.grid(2)
    a = scale(i/(WIDTH-1), MIN_X, MAX_X)
    b = scale(j/(HEIGHT-1), MIN_Y, MAX_Y)
    x = y = 0
    d = 1e9
    rot = 1/math.sqrt(2)
    pi_const = math.sqrt(2)*math.pi
    for _ in range(max_iter):
        xx = x*x
        yy = y*y
        t_x = xx - yy + a
        t_y = (x+x)*y + b
        x = t_x
        y = t_y
        # p1_x = -0.75
        # p1_y = 0.25
        # p2_x = -0.75
        # p2_y = -0.25
        # p3_x = .25
        # p3_y = 0
        u_x = x + math.pi/2 + t
        u = u_x*rot-y*rot
        v = y*rot+u_x*rot
        # d = min(d, abs((x-p1_x)**2 + (y-p1_y)**2))
        # d = min(d, abs((x-p2_x)**2 + (y-p2_y)**2))
        # d = min(d, abs((x-p3_x)**2 + (y-p3_y)**2))
        # d = min(d, abs((u % pi_const)))
        # d = min(d, abs((v % pi_const)))

        d = min(d, abs((u % pi_const)-pi_const))
        d = min(d, abs((v % pi_const)-pi_const))

        # d = min(d, abs(math.sin(x-1.5) + math.cos(y)))
        # d = min(d, abs(math.sqrt(abs(math.sin(x))) + math.cos(y)))
    arr[j, i] = d

In [25]:
def build_fractal(cuda_kernel, kernel_params, post_process, file_name, height=HEIGHT, width=WIDTH, dpi=DPI, imshow_params={"cmap":"plasma_r", "aspect":"auto"}):
    canvas = cp.empty((height, width), dtype=np.float32)
    threadsperblock = (16, 16)
    blockspergrid_x = math.ceil(canvas.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(canvas.shape[1] / threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)


    cuda_kernel[blockspergrid, threadsperblock](canvas, *kernel_params)

    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig_size=(width/dpi, height/dpi)
    fig.set_size_inches(fig_size)
    fig.add_axes(ax)
    
    host_arr = cp.asnumpy(post_process(canvas))
    ax.imshow(host_arr, **imshow_params)

    plt.savefig(file_name, dpi=dpi)

In [27]:
"""Render classic mandelbrot"""

def post_process(arr):
    glow = 1
    arr = cp.log(arr+glow)
    arr = (arr - arr.min()) /(arr.max()-arr.min())
    return arr

build_fractal(cuda_generate_orbit_trap_fractal, (500,0), post_process, "local_artifacts/test_mandelbrot.png")

In [29]:
"""Render orbit trap"""

def post_process(arr):
    glow = 0.0025
    arr = cp.log(arr+glow)
    arr = (arr - arr.min()) /(arr.max()-arr.min())
    return arr

build_fractal(cuda_generate_orbit_trap_fractal, (3000,0), post_process, "local_artifacts/_fractal.png")

In [ ]:
"""Render videos (WIP)"""
canvas = cp.empty((HEIGHT, WIDTH), dtype=np.float32)
threadsperblock = (16, 16)
blockspergrid_x = math.ceil(canvas.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(canvas.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
NUM_FRAMES=120
DPI = 500

for t in range(NUM_FRAMES):
    cuda_generate_orbit_trap_fractal[blockspergrid, threadsperblock](canvas, 1000, t/(NUM_FRAMES-1))

    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    FIG_SIZE=(WIDTH/DPI, HEIGHT/DPI)
    fig.set_size_inches(FIG_SIZE)
    fig.add_axes(ax)
    glow = 0.0025
    out = cp.log(canvas+glow)
    out = (out - out.min()) /(out.max()-out.min())
    host_arr = cp.asnumpy(out)
    ax.imshow(host_arr, cmap='plasma_r', aspect='auto')

#plt.show()
plt.savefig("local_artifacts/_fractal.png", dpi=DPI)
print("Done!")

In [6]:
"""Trim existing images"""

from PIL import Image

# Kept large because renders tend to be huge
Image.MAX_IMAGE_PIXELS = 933120000

title = "lattice_cursed"
im = Image.open(f"local_artifacts/{title}.png")
 
TRIM_WIDTH, TRIM_HEIGHT = 2560, 1440

im1 = im.resize((TRIM_WIDTH, TRIM_HEIGHT))
im1.save(f"local_artifacts/{title}_trim.png")
# Shows the image in image viewer
# im1.show()